In [ ]:

"""
Effective Power (EP) is calculated as EP = power * STAB * enemy_type_effect1 * enemy_type_effect2 * burn_effect * weather_effect
power: base power of the move
STAB: 1.5 if move used is same type as user. Otherwise, it is 1.
enemy_type_effect1: .5 if the move used is not very effective. Otherwise, it is 1
enemy_type_effect2: .5 if the move used is not very effective. Otherwise, it is 1
burn_effect: if user is burned and the attack if physical, this is .5. Otherwise, it is 1.
weather_effect: if the weather is sunny and the attack is fire or rainy and the attack is water, this is 1.5
                if the weather is sunny and the attack is water or rainy and the attack is fire, this is .5

"""

In [37]:

import requests
import json

def damageCalc(atk_pkmn, target_pkmn, move, burn = False, weather = None):
    #calculates the base power of the move
    response = requests.get(f'https://pokeapi.co/api/v2/move/{move}')
    move_info = json.loads(response.text)
    power_move = move_info['power']
    if power_move is None:
        return None
    type_move = move_info['type']['name']

    move_name = move_info['name']

    stab = calculateStab(atk_pkmn,type_move)
    target_type1, target_type2 = getPkmnTypes(target_pkmn)   
    enemy_type_effect1 = calculateDamageMultiplier(type_move, target_type1)
    if target_type2 is not None:
        enemy_type_effect2 = calculateDamageMultiplier(type_move, target_type2)
    else:
        enemy_type_effect2 = 1
    burn_effect = calcBurnMultiplier(burn, move_info)
    weather_effect = weatherFunc(weather, move_name)
    print("power_move: ", power_move)
    print("stab: ", stab)
    print("enemy_type_effects: ", enemy_type_effect1 * enemy_type_effect2)
    print("burn_effect: ", burn_effect)
    print("weather_effect: ", weather_effect)
    EP = power_move * stab * enemy_type_effect1 * enemy_type_effect2 * burn_effect * weather_effect    
    return EP

def calcBurnMultiplier(is_burned, move_info):
    damage_class = move_info["damage_class"]
    effectiveness = damage_class["name"]
    if effectiveness == "physical" and is_burned:
        return .5
    return 1
 
def getPkmnTypes(pokemon_name):
    response = requests.get(f'https://pokeapi.co/api/v2/pokemon/{pokemon_name}')
    assign = json.loads(response.text)
    type_info_list = assign['types']

    first_move_info = type_info_list[0]
    first_dict_info = first_move_info['type'] 
    type_name1 = first_dict_info['name'] 
    try:
        second_move_info = type_info_list[1]
        second_dict_info = second_move_info['type'] 
        type_name2 = second_dict_info['name'] 
        return type_name1, type_name2
    except IndexError:
        return type_name1, None

def calculateStab(atk_pkmn, type_move):
    type_name1, type_name2 = getPkmnTypes(atk_pkmn)

    if type_name1 == type_move or type_name2 == type_move:
        return 1.5
    return 1

def calculateDamageMultiplier(move_type, defending_pkm_type):
    response = requests.get(f'https://pokeapi.co/api/v2/type/{move_type}')
    type_info = json.loads(response.text)
    effectiveness_chart = type_info['damage_relations']
        
    no_damage_types = [elt['name'] for elt in effectiveness_chart["no_damage_to"]]
    half_damage_types = [elt['name'] for elt in effectiveness_chart["half_damage_to"]]
    double_damage_types = [elt['name'] for elt in effectiveness_chart["double_damage_to"]]
    
    if defending_pkm_type in no_damage_types:
        return 0
    if defending_pkm_type in half_damage_types:
        return 0.5
    if defending_pkm_type in double_damage_types:
        return 2
    return 1


def weatherFunc(weather_effect, move_name):
    response = requests.get(f'https://pokeapi.co/api/v2/move/{move_name}')
    move_list = json.loads(response.text)
    move_class = move_list['type']
    
    if ((weather_effect in ['sun',  'sunny'] and move_class['name'] == 'fire')  or
        (weather_effect in ['rain', 'rainy'] and move_class['name'] == 'water')):
        return 1.5    
    if ((weather_effect  in ['rain', 'rainy'] and move_class['name'] == 'fire') or
        (weather_effect  in ['sun',  'sunny'] and move_class['name'] == 'water')):
        return .5
    return 1 


In [52]:
import random
number = random.randint(1,1010)
response1 = requests.get(f'https://pokeapi.co/api/v2/pokemon/{number}')
pokemon_json = json.loads(response1.text)
pokemon1 = pokemon_json['name']

number = random.randint(1,1010)
response2 = requests.get(f'https://pokeapi.co/api/v2/pokemon/{number}')
pokemon2 = json.loads(response2.text)['name']

num_moves = len(pokemon_json['moves'])
number = random.randint(0, num_moves-1)
move = pokemon_json['moves'][number]['move']['name']

is_burned = bool(random.randint(0,1))

weather_type = ['sun','rain',None][random.randint(0,2)]
print(pokemon1, pokemon2, move, is_burned, weather_type)


duraludon natu rock-slide True None


In [46]:
print(damageCalc(pokemon1, pokemon2, move, burn = is_burned, weather = weather_type))

power_move:  50
stab:  1.5
enemy_type_effects:  0.5
burn_effect:  0.5
weather_effect:  1
18.75


In [50]:
def compareMoves(atkp, defp, move1, move2, burned = False, weather_type = None):
    dmg1 = damageCalc(atkp, defp, move1, burn = burned, weather = weather_type)
    dmg2 = damageCalc(atkp, defp, move2, burn = burned, weather = weather_type)
    return move1 if dmg1 > dmg2 else move2

In [55]:
print(compareMoves('scizor', 'amoonguss', 'bullet-punch', 'u-turn', burned = False, weather_type = None))

power_move:  40
stab:  1.5
enemy_type_effects:  1
burn_effect:  1
weather_effect:  1
power_move:  70
stab:  1.5
enemy_type_effects:  1.0
burn_effect:  1
weather_effect:  1
u-turn
